部分同步：系统最终会表现得像同步系统，但事先不知道什么时候开始同步，也不知道具体时间界限是多少。

部分同步模型的经典定义：

1. 模型A:

未知上界，但一直存在：消息延迟$\leq \Delta$，处理时间$\leq \Phi$。系统始终是同步的，只是算法不知道上界

2. 模型B:

已知上界，但在未知时间后才成立：存在一个全局稳定时间(GST)，在GST之前：任意延迟，丢包、慢节点；在GST之后：消息延迟$\leq \Delta$，处理时间$\leq \Phi$，上界已知。系统最终变得同步。

部分同步下可以实现：一致性、有效性、终止性

部分同步一致性的核心机制：
1. 轮次：系统被划分为逻辑轮次：每一轮有一个leader，节点在一轮内等待消息，超时则进入下一轮
2. 超时：超时=猜测系统已经同步；超时过短则频繁换leader，超时过长则响应慢
3. Leader轮换：如果当前leader不工作：节点超时，切换到下一个leader，最终会轮到一个正确的leader

PSynchFD算法：

模型假设：消息延迟$\leq \Delta$，处理时间$\leq \Phi$。

进程只会crash，无拜占庭故障，消息不丢失不乱序。如果一个进程在时间$\Delta +\Phi$，那它一定已经崩溃。

```plaintxt
constants:
  Δ  // 消息延迟上界
  Φ  // 处理时间上界
  T = Δ + Φ

state:
  Suspect_p = ∅

every round r do
  send <HEARTBEAT, r> to all processes

  wait until end of round r (time T)

  for each process q ≠ p do
    if no <HEARTBEAT, r> from q received then
       Suspect_p ← Suspect_p ∪ {q}
```

A'算法：不是工程算法，而是一个理论构造，用于证明：在完全同步系统中，可以实现完美故障检测器 P（PSynchFD）

系统假设：
1. 系统模型：进程集合：$\Pi={p_1...p_n}$，故障类型为crash，网络不丢包、不重复、不乱序
2. 同步假设：已知上界：消息延迟$\leq \Delta$，处理时间$\leq \Phi$。并且这些上界：从时间0开始始终成立，所有进程都知道

```plaintxt
constants:
  Δ   // 消息传输上界
  Φ   // 处理时间上界
  T = Δ + Φ

state:
  suspected_p = ∅

for round r = 1, 2, 3, ... do
  // Step 1: 广播存活消息
  send <ALIVE, r> to all processes

  // Step 2: 等待一个完整轮次
  wait T time units

  // Step 3: 判断失败进程
  for each process q ≠ p do
    if no <ALIVE, r> from q received then
        suspected_p ← suspected_p ∪ {q}
```

A′ 算法是在完全同步系统中，通过已知的消息与处理时间上界，采用周期性存活广播与固定轮次超时机制，实现满足强准确性与强完整性的完美故障检测器，从而为一致性问题的可解性提供构造性证明。

PSynchAgreement算法：证明：在完全同步系统中，只要能完美检测故障，一致性问题就变得“结构上简单”

系统假设：
1. 进程与故障：进程集合：$\Pi={p_1...p_n}$，故障类型为crash，每个进程有一个输入值$v_p$
2. 同步与故障检测：已知上界：消息延迟$\leq \Delta$，处理时间$\leq \Phi$。系统提供 PSynchFD，即：不会误判正确进程；所有崩溃进程最终被所有正确进程发现

进程p的行为：
```plaintxt
// Phase 1: 广播提议
broadcast <PROPOSE, v_p> to all processes

// Phase 2: 等待存活进程的提议
wait until:
  for every process q ∉ FD_p :
    a <PROPOSE, v_q> has been received

// Phase 3: 决定
decide min { v_q | q ∉ FD_p }
```